In [10]:
"""PyAudio Example: Play a wave file."""
import wave, pyaudio
import os
os.environ['SDL_AUDIODRIVER'] = 'dsp'
def play_wav(wav_name, CHUNK=1024):
	with wave.open(wav_name, 'rb') as wf:
		p = pyaudio.PyAudio()
		print(wf.getsampwidth())
		print(wf.getnchannels())
		print(wf.getframerate())
		stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
						channels=wf.getnchannels(),
						rate=wf.getframerate(),
						output=True)
		while len(data := wf.readframes(CHUNK)):
			stream.write(data)
		stream.close()
		p.terminate()

In [105]:
play_wav("voice/short.wav")

ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib confmisc.c:1369:(snd_func_refer) Unable to find definition 'cards.0.pcm.hdmi.0:CARD=0,AES0=4,AES1=130,AES2=0,AES3=2'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM hdmi
ALSA lib confmisc.c:1369:(snd_func_refer) Unable to find definition 'cards.0.pcm.hdmi.0:CARD=0,AES0=4,AES1=130,AES2=0,AES3=2'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2

2
2
44100


In [19]:
import soundfile as sf
data, samplerate = sf.read('voice/short.wav', dtype='float32')
len(data) / samplerate, data.shape

(3.1956462585034013, (140928, 2))

In [18]:
with wave.open('voice/short.wav', 'rb') as wf:
    data_block = wf.readframes(1024)

In [36]:
import struct

# Input binary data
binary_data = b'\x40\x49\x0f\xdb'
# Unpack the binary data as a 32-bit float
unpacked_float = struct.unpack('f', binary_data)[0]

print(unpacked_float)

-4.03314608963584e+16


In [50]:
import struct

data[0,0], struct.unpack("<l", data_block[:4])

(-0.059020996, (-126683022,))

In [55]:
data_block[:4]

b'r\xf8r\xf8'

In [74]:
with open('voice/short.wav', 'rb') as f:
    header = f.read(44)

In [11]:
class WavInfo:
    """Class for storing wav parameters to pass to pyaudio"""
    frame_rate: int
    num_channels: int
    sample_width: int

    def __init__(self, frame) -> float:
        print(frame)
        assert(len(frame)==44)
        assert(frame[0:4]==b'RIFF')
        wav_size = struct.unpack('I', frame[4:8])[0]
        assert(frame[8:12]==b'WAVE')
        assert(frame[12:16]==b'fmt ')
        assert(struct.unpack('I', frame[16:20])[0] == 16)
        format_tag, num_channels = struct.unpack('2h', frame[20:24])
        sample_rate, byte_rate = struct.unpack('2I', frame[24:32])
        block_align, bits_per_sample = struct.unpack('2h', frame[32:36])
        assert(byte_rate == block_align * sample_rate)
        assert(frame[36:40]==b'data')
        data_size = struct.unpack('I', frame[40:44])[0]
        assert(wav_size == data_size + 36)

        self.sample_width = bits_per_sample // 8
        self.frame_rate = sample_rate
        self.num_channels = num_channels
        # num_samples = data_size // (num_channels * sample_width)

import wave, pyaudio
import os, struct
os.environ['SDL_AUDIODRIVER'] = 'dsp'

def play_wav2(wav_name, CHUNK=1024):
    p = pyaudio.PyAudio()

    stream = None
    with open(wav_name, 'rb') as wf:
        while (len(block := wf.read(CHUNK))):
            if stream is None:
                header = WavInfo(block[:44])
                stream = p.open(format=p.get_format_from_width(header.sample_width),
						channels=header.num_channels,
						rate=header.frame_rate,
						output=True)
                data = block[44:]
            else:
                data = block
            stream.write(data)
        stream.close()
    p.terminate()

play_wav2("voice/short.wav")

ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib confmisc.c:1369:(snd_func_refer) Unable to find definition 'cards.0.pcm.hdmi.0:CARD=0,AES0=4,AES1=130,AES2=0,AES3=2'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM hdmi
ALSA lib confmisc.c:1369:(snd_func_refer) Unable to find definition 'cards.0.pcm.hdmi.0:CARD=0,AES0=4,AES1=130,AES2=0,AES3=2'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2

b'RIFF$\x9a\x08\x00WAVEfmt \x10\x00\x00\x00\x01\x00\x02\x00D\xac\x00\x00\x10\xb1\x02\x00\x04\x00\x10\x00data\x00\x9a\x08\x00'


In [17]:
with wave.open('voice/short.wav', 'rb') as wf:
	total_len = 0
	while len(data := wf.readframes(1024)):
		total_len += len(data)
total_len // 4

140928

In [4]:
samplerate = 44100
# A note on the left channel for 1 second.
t = np.linspace(0, 1, samplerate)
left_channel = 0.5 * np.sin(2 * np.pi * 440.0 * t)
# Noise on the right channel.
right_channel = np.random.random(size=samplerate)
# Put the channels together with shape (2, 44100).
audio = np.array([left_channel, right_channel]).T
# Convert to (little-endian) 16 bit integers.
audio = (audio * (2 ** 15 - 1)).astype("<h")
with wave.open("sound1.wav", "w") as f:
    # 2 Channels.
    f.setnchannels(2)
    # 2 bytes per sample.
    f.setsampwidth(2)
    f.setframerate(samplerate)
    f.writeframes(audio.tobytes())

b'r\xf8r\xf8\x04\xfb\x04\xfb\x0f\xff\x0f\xffV\x02V\x02\x1a\x01\x1a\x01\xf2\x00\xf2\x00:\x01:\x01\xd0\x00\xd0\x00\x80\xff\x80\xff6\xfb6\xfb\xbd\xf9\xbd\xf9\x02\xf8\x02\xf8\x11\xf6\x11\xf6,\xf2,\xf2\xd9\xf5\xd9\xf5\x02\xfa\x02\xfa\xf9\xf7\xf9\xf7r\xf9r\xf9+\xf7+\xf7e\xf4e\xf4c\xf9c\xf9\x0e\xfe\x0e\xfe\xf0\xf8\xf0\xf8\xe5\xf5\xe5\xf5\x87\xf4\x87\xf4\x82\xf1\x82\xf1\xb1\xf1\xb1\xf1=\xf1=\xf1\x98\xf1\x98\xf1k\xf2k\xf2\x00\xf5\x00\xf5\xa4\xf8\xa4\xf88\xfa8\xfaF\xf8F\xf8r\xf5r\xf5l\xf0l\xf0\x01\xed\x01\xed\x95\xef\x95\xef9\xf49\xf4\xaa\xfa\xaa\xfaZ\xffZ\xff\xfc\x00\xfc\x00K\xf7K\xf7\x9f\xed\x9f\xed:\xed:\xed\x8d\xeb\x8d\xeb3\xea3\xea\x87\xe8\x87\xe8\xe6\xe9\xe6\xe9\x98\xec\x98\xecD\xeeD\xee\xf7\xec\xf7\xec\xe0\xe6\xe0\xe6Y\xe0Y\xe0\x81\xde\x81\xdex\xe3x\xe3\x03\xe8\x03\xe8x\xebx\xeb\x9d\xed\x9d\xed\x1e\xef\x1e\xefj\xefj\xef\x9b\xf0\x9b\xf0\xe4\xf3\xe4\xf3\xff\xf0\xff\xf0\n\xee\n\xee\xaf\xee\xaf\xee\xce\xed\xce\xed\x1b\xea\x1b\xea\xf8\xe8\xf8\xe8\xfb\xeb\xfb\xeb\xac\xeb\xac\xeb\xc2\xe8\xc2\xe8

In [24]:
import torchaudio

sent_file = '/home/zipa/Documents/kapitza_bot/audio-tcp/voice/1.wav'
rcvd_file = '/home/zipa/Documents/kapitza_bot/.received/ay7slazh.wav'

data1 = open(sent_file, 'rb').read()
data2 = open(rcvd_file, 'rb').read()
# assert(data1[:98092] == data2[:98092])
len(data1), len(data2)

(98092, 1024)

In [28]:
struct.decode(b'\x80\x80\x7f\x80\x80\x7f\x80\x7f')

AttributeError: module 'struct' has no attribute 'decode'

b'RIFF$\x7f\x01\x00WAVEfmt \x10\x00\x00\x00\x01\x00\x01\x00\xc0]\x00\x00\xc0]\x00\x00\x01\x00\x08\x00data\x00\x7f\x01\x00\x80\x7f\x80\x80\x7f\x80\x7f\x7f\x7f\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x7f\x80\x80\x80\x80\x80\x80\x7f\x7f\x80\x80\x80\x7f\x7f\x80\x80\x7f\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x80\x8

AssertionError: 

In [27]:
# re-encoding with torchaudio
# generates noises
import torchaudio

data = torchaudio.load('voice/3.wav')
torchaudio.save('voice/3.wav', data[0], data[1], encoding="PCM_U", backend="soundfile")
play_wav('voice/3.wav')

ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib confmisc.c:1369:(snd_func_refer) Unable to find definition 'cards.0.pcm.hdmi.0:CARD=0,AES0=4,AES1=130,AES2=0,AES3=2'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM hdmi
ALSA lib confmisc.c:1369:(snd_func_refer) Unable to find definition 'cards.0.pcm.hdmi.0:CARD=0,AES0=4,AES1=130,AES2=0,AES3=2'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2

1
1
24000


In [16]:
torchaudio.__version__

'2.4.1+cu124'

Error: unknown format: 3